## Normalization Ingredients

Noah Meissner 11.06.2025

In [1]:
import pandas as pd
from data_structure.paths import RAW_SET, NER_EVAL
from DataLoader_Ingredients import DataLoader
from request.recipe_api import request_api
from normalizer.normalizer import Normalization
from loader.load_normalization import save_response
from data_structure.model_name import ModelName
from data_structure.DataType import DataType
from tqdm import tqdm
import re

/Users/noah/Documents/github/MultiAgentBiase/Preprocessing/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Take Dataset
- The following function loads the dataset and gives unique ingreidents as test and train split
- Train 50000
- Test 1000

In [2]:
df_whole = pd.read_csv(RAW_SET)
loader_obj = DataLoader(df_whole)
train, test = loader_obj.get_set()

### Analysis
we now analyse our approach, using Normalization

In [3]:
#TODO Grafik wie er funktioniert

In [4]:
def api_request(txt):
    output = request_api(txt)
    detailed_info = output.get("detailed_info", [])
    
    if detailed_info and len(detailed_info) > 0 and detailed_info[0] and len(detailed_info[0]) > 0:
        erkannte_zutat = [detailed_info[0][0].get("erkannteZutat", "")]
    else:
        erkannte_zutat = [] 
    
    return erkannte_zutat

In [5]:
def clean_ingredients(txt):
    if txt is None:
        return ""
    txt = txt.replace("\n","").replace("\t","").replace("%x22","").strip()
    txt = re.sub(r"\s+"," ", txt)
    return txt

In [6]:
normalisation = Normalization()
ls_res = []
equal = 0
under = 0
over = 0
zero = 0
ls_api = []
for ingredient in tqdm(test, desc="Normalize Data"):
    ingredient = clean_ingredients(ingredient)
    original_class = api_request(ingredient)
    pipeline_ingredient, len_ingredients = normalisation.normalize(ingredient)
    # Calc Stats for ONLY API
    if len(original_class) > 0 and original_class[0] != "Nicht erkannt":
        equal += int(len(original_class) == len_ingredients)
        under += int(len(original_class) < len_ingredients)
        over += int(len(original_class) > len_ingredients)
    zero += int(len(original_class) == 0)
    ############
    ls_res.append(pipeline_ingredient)
    ls_api.append({ingredient : original_class})
save_response(ModelName.UNI,DataType.eval,ls_api)
save_response(ModelName.Pipeline,DataType.eval,ls_res)

/Users/noah/Documents/github/MultiAgentBiase/Preprocessing/model/modern_bert/model.pt


Normalize Data: 100%|██████████| 1000/1000 [15:29<00:00,  1.08it/s]


In [7]:
{"equal": equal, "under": under, "over": over, "zero":zero}


{'equal': 345, 'under': 12, 'over': 4, 'zero': 6}

In [8]:
normalisation.get_stats()

{'equal': 679, 'under': 321, 'over': 0, 'zero': 321}

In [ ]:
Pipeline_wrong = 50

In [2]:
norm = Normalization()
norm.tokenize("200g Mehl")

/Users/noah/Documents/github/MultiAgentBiase/Preprocessing/model/modern_bert/model.pt


Compiling the model with `torch.compile` and using a `torch.cpu` device is not supported. Falling back to non-compiled mode.


[['200', 'Number'], ['g', 'Units'], [' Mehl', 'Ingredients']]
{'original': '200g Mehl', 'ner': {'200g Mehl': {'entities': [['200', 'Number'], ['g', 'Units'], [' Mehl', 'Ingredients']]}}}
